# Market Basket Analysis

Técnica no supervisada para encontrar reglas en transacciones. Los datasets son conjuntos de cestas con items comprados. Requiere codificacion binaria (uno por producto).

# Algoritmo Apriori

Como funciona:

Ingresa una tabla binaria donde las filas son transacciones y las columnas items. Luego se calcula el soporte de cada item (frecuencia relativa). Se generan combinaciones frecuentes entre los items que sobrepasan un umbral impuesto para el soporte. Se itera el proceso hasta encontrar todo los conjuntos de items que cumplen con el soporte y se procede a crear las reglas. Luego se calcula las confianzas de cada regla y las que cumplan con un umbral impuesto igualmente para la confianza se toman en cuenta para, por ultimo, calcular el lift de este ultimo conjunto de reglas.

El lift es cuan mas probable es que ocurran juntos el antecedentes y el consecuente de una regla, comparado con que ocurran de forma independiente.

Las consideraciones para el Lift son:

- Lift = 1 : A y B son independientes
- Lift > 1 : A y B ocurren juntos más frecuentemente de lo esperado
- Lift < 1 : A y B ocurren juntos menos de lo esperado

Carguemos las librerías necesarias para la realización del modelo.

In [ ]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.preprocessing import TransactionEncoder

Primero haremos un dataset pequeño de ejemplo.

In [ ]:
dataset = [['leche','pan','mantequilla'],
           ['pan','cerveza','huevo'],
           ['leche','pan','mantequilla','huevo'],
           ['pan','mantequilla'],
           ['leche','pan','huevo']
           ]

Lo primero que haremos es transformar la canasta de compra que es un dataset con listas de elementosa un dataset binario

Preprocesamiento.

In [ ]:
te = TransactionEncoder()
te_ary = te.fit(dataset).transform(dataset)
df = pd.DataFrame(te_ary, columns=te.columns_)

Veamos el dataframe resultante.

In [ ]:
df

,cerveza,huevo,leche,mantequilla,pan
0,False,False,True,True,True
1,True,True,False,False,True
2,False,True,True,True,True
3,False,False,False,True,True
4,False,True,True,False,True


Ahora vamos a calcular los items frecuentes a través del algoritmo Apriori.

In [ ]:
frequent_itemsets = apriori(df, min_support=0.5, use_colnames=True)

In [ ]:
frequent_itemsets

,support,itemsets
0,0.6,(huevo)
1,0.6,(leche)
2,0.6,(mantequilla)
3,1.0,(pan)
4,0.6,"(huevo, pan)"
5,0.6,"(leche, pan)"
6,0.6,"(pan, mantequilla)"


Ahora las reglas de asociaciones las obtendremos de la siguiente forma.

In [ ]:
rules = association_rules(frequent_itemsets,
                          metric="lift",
                          min_threshold=1)

/usr/local/lib/python3.11/dist-packages/mlxtend/frequent_patterns/association_rules.py:186: RuntimeWarning: invalid value encountered in divide
  cert_metric = np.where(certainty_denom == 0, 0, certainty_num / certainty_denom)


Visualicemos las reglas.

In [ ]:
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
0,(huevo),(pan),0.6,1.0,0.6,1.0,1.0,1.0,0.0,inf,0.0,0.6,0.0,0.8
1,(pan),(huevo),1.0,0.6,0.6,0.6,1.0,1.0,0.0,1.0,0.0,0.6,0.0,0.8
2,(leche),(pan),0.6,1.0,0.6,1.0,1.0,1.0,0.0,inf,0.0,0.6,0.0,0.8
3,(pan),(leche),1.0,0.6,0.6,0.6,1.0,1.0,0.0,1.0,0.0,0.6,0.0,0.8
4,(pan),(mantequilla),1.0,0.6,0.6,0.6,1.0,1.0,0.0,1.0,0.0,0.6,0.0,0.8
5,(mantequilla),(pan),0.6,1.0,0.6,1.0,1.0,1.0,0.0,inf,0.0,0.6,0.0,0.8


Mostremos los resultados de una forma más ordenada.

In [ ]:
print("\n--- Items Frecuentes ---")
print(frequent_itemsets)

print("\n--- Reglas de Asociacion ---")
print(rules[['antecedents','consequents','support','confidence','lift']])


--- Items Frecuentes ---
   support            itemsets
0      0.6             (huevo)
1      0.6             (leche)
2      0.6       (mantequilla)
3      1.0               (pan)
4      0.6        (huevo, pan)
5      0.6        (leche, pan)
6      0.6  (pan, mantequilla)

--- Reglas de Asociacion ---
     antecedents    consequents  support  confidence  lift
0        (huevo)          (pan)      0.6         1.0   1.0
1          (pan)        (huevo)      0.6         0.6   1.0
2        (leche)          (pan)      0.6         1.0   1.0
3          (pan)        (leche)      0.6         0.6   1.0
4          (pan)  (mantequilla)      0.6         0.6   1.0
5  (mantequilla)          (pan)      0.6         1.0   1.0


Ahora lo que haremos es utilizar una lista de compras de un supermercado.

In [ ]:
import urllib.request

# URL RAW del archivo CSV en GitHub
url_raw = "https://raw.githubusercontent.com/stedy/Machine-Learning-with-R-datasets/master/groceries.csv"

Abrimos la URL con urllib y leemos byte a byte.

In [ ]:
response = urllib.request.urlopen(url_raw)

In [ ]:
# Armamos una lista vacia
transacciones = []
# Y ahora linea por linea de lo que está en response vamos llenando la lista
for linea in response:
    fila = linea.decode('utf-8').strip() # ej. "whole milk,rolls/buns,soda"
    items = fila.split(',')              # ej. ["whole milk", "rolls/buns", "soda"]
    transacciones.append(items)

Visualizacion de las primeras 10 listas.

In [ ]:
transacciones[0:10]

[['citrus fruit', 'semi-finished bread', 'margarine', 'ready soups'],
 ['tropical fruit', 'yogurt', 'coffee'],
 ['whole milk'],
 ['pip fruit', 'yogurt', 'cream cheese ', 'meat spreads'],
 ['other vegetables',
  'whole milk',
  'condensed milk',
  'long life bakery product'],
 ['whole milk', 'butter', 'yogurt', 'rice', 'abrasive cleaner'],
 ['rolls/buns'],
 ['other vegetables',
  'UHT-milk',
  'rolls/buns',
  'bottled beer',
  'liquor (appetizer)'],
 ['pot plants'],
 ['whole milk', 'cereals']]

In [ ]:
# Creamos un TransactionEncoder
te = TransactionEncoder()

# Ajustamos el codificador a nuestra lista de transacciones
te_ary = te.fit(transacciones).transform(transacciones)

# Convertimos el array en un DataFrame
df_binario = pd.DataFrame(te_ary, columns=te.columns_)

In [ ]:
df_binario.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,baby food,bags,baking powder,bathroom cleaner,beef,...,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,True,False,False


Frecuencias y Reglas obtenidas.

In [ ]:
freq_itemsets = apriori(df_binario, min_support=0.01, use_colnames=True)
rules = association_rules(freq_itemsets, metric="lift", min_threshold=1.5)

Por último, procedamos a mostrar los resultado.

In [ ]:
print("\n--- Items Frecuentes ---")
print(freq_itemsets)

print("\n--- Reglas de Asociacion ---")
print(rules[['antecedents','consequents','support','confidence','lift']])


--- Items Frecuentes ---
      support                                       itemsets
0    0.033452                                     (UHT-milk)
1    0.017692                                (baking powder)
2    0.052466                                         (beef)
3    0.033249                                      (berries)
4    0.026029                                    (beverages)
..        ...                                            ...
327  0.011998  (root vegetables, whole milk, tropical fruit)
328  0.014540          (yogurt, root vegetables, whole milk)
329  0.010473                     (yogurt, whole milk, soda)
330  0.015150           (yogurt, whole milk, tropical fruit)
331  0.010880       (yogurt, whole milk, whipped/sour cream)

[332 rows x 2 columns]

--- Reglas de Asociacion ---
                          antecedents                       consequents  \
0                              (beef)                (other vegetables)   
1                  (other vegetables) 

Ahora queremos ver las reglas ordenadas por Lift.

In [ ]:
print(rules[['antecedents','consequents','support','confidence','lift']].sort_values(by='lift', ascending=False).head(10))

                            antecedents                         consequents  \
244                              (curd)                (yogurt, whole milk)   
241                (yogurt, whole milk)                              (curd)   
224    (other vegetables, citrus fruit)                   (root vegetables)   
225                   (root vegetables)    (other vegetables, citrus fruit)   
330          (yogurt, other vegetables)                (whipped/sour cream)   
335                (whipped/sour cream)          (yogurt, other vegetables)   
291                   (root vegetables)  (other vegetables, tropical fruit)   
290  (other vegetables, tropical fruit)                   (root vegetables)   
3                                (beef)                   (root vegetables)   
2                     (root vegetables)                              (beef)   

      support  confidence      lift  
244  0.010066    0.188931  3.372304  
241  0.010066    0.179673  3.372304  
224  0.010371   